In [1]:
# Import necessary libraries and modules
from langchain.chat_models import ChatOpenAI
import sqlite3
import pandas as pd
import os

# Set the OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the Langchain ChatOpenAI model
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4")

# Connect to the SQLite database
connection = sqlite3.connect("Chinook_Sqlite.sqlite")

c:\Users\Manu\Data_Science\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
# Import necessary libraries and modules
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

# Create an instance of SQLDatabase using the 'customer.db' SQLite database
db = SQLDatabase.from_uri('sqlite:///Chinook_Sqlite.sqlite')

# Create an SQL agent executor with specified parameters
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
    return_intermediate_steps=True
)

# Define user input
#user_inquiry = "What is the the total sales per country?"

# Run the agent to generate a response
#agent_executor.run(user_inquiry)

In [5]:
#import streamlit as st
from langchain import LLMChain
from langchain.agents import (AgentExecutor, Tool, ZeroShotAgent)
from langchain_experimental.tools import PythonREPLTool

# Define a description to suggest how to determine the choice of tool
description = (
    "Useful when you require to answer analytical questions. "
    "Use this more than the Python REPL tool if the question is about analytics,"
    "like 'How many tracks per genre are there?' or 'count the number of customers by genre'. "
    "Try not to use clause in the SQL."
)

# Create a Tool object for customer data with the previously defined agent executor 'create_sql_agent' and description
analytics_tool = Tool(
    name="Charts",
    func=agent_executor.run,
    description=description,
)

# Create the whole list of tools
tools = [PythonREPLTool()]
tools.append(analytics_tool)

# Define the prefix and suffix for the prompt
prefix = "Below are tools that you can access:"
suffix = (
    "Pass the relevant part of the request directly to the Charts tool.\n\n"
    "Request: {input}\n"
    "{agent_scratchpad}"
)

# Create the prompt using ZeroShotAgent
# Use agent_scratchpad to store the actions previously used, guiding the subsequent responses.
agent_prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

# Create an instance of ZeroShotAgent with the LLMChain and the allowed tool names
zero_shot_agent = ZeroShotAgent(
    llm_chain=LLMChain(llm=llm, prompt=agent_prompt),
    allowed_tools=[tool.name for tool in tools]
)

# Create an AgentExecutor which enables verbose mode and handling parsing errors
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=zero_shot_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

# Define Streamlit layout
#st.title("SQL Analytics with LangChain and Streamlit")

# Define user input
#user_inquiry = "What are the total sales per country?"

user_inquiry = "Use a bar graph to visualize the result of the following inquiry: " \
          "What are the total sales per country?"

# Run the agent to generate a response
agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
Question: What are the total sales per country?
Thought: I need to get the total sales for each country. I can achieve this by summing the sales and grouping by the country.
Action: Analytics
Action Input: 
```
SELECT country, SUM(sales) as Total_Sales
FROM sales
GROUP BY country
```

> Entering new AgentExecutor chain...
Question: What is the total sales for each country?
Thought: The request is asking for the total sales for each country. This is a typical aggregation task which can be done using SQL. 
Action: Analytics
Action Input: ```
SELECT country, SUM(sales) as Total_Sales
FROM sales
GROUP BY country
```

> Entering new AgentExecutor chain...
Question: What is the total sales per country?
Thought: The request is already a SQL query, so I just need to pass it directly to the Analytics tool.
Action: Analytics
Action Input: 
```
SELECT country, SUM(sales) as Total_Sales
FROM sales
GROUP BY country
```

> Entering new SQL Agent Executor chain

'The total sales for each country are as follows:\n\n- Argentina: 37.62\n- Australia: 37.62\n- Austria: 42.62\n- Belgium: 37.62\n- Brazil: 190.10\n- Canada: 303.96\n- Chile: 46.62\n- Czech Republic: 90.24\n- Denmark: 37.62\n- Finland: 41.62\n- France: 195.10\n- Germany: 156.48\n- Hungary: 45.62\n- India: 75.26\n- Ireland: 45.62\n- Italy: 37.62\n- Netherlands: 40.62\n- Norway: 39.62\n- Poland: 37.62\n- Portugal: 77.24\n- Spain: 37.62\n- Sweden: 38.62\n- USA: 523.06\n- United Kingdom: 112.86'

In [6]:
from langchain.memory import ConversationBufferMemory

# Skipped here - Define your own prefix, suffix, and description with "chat_history" for the prompt
# Keep the original list of tools

# Create the prompt using ZeroShotAgent with additonal "chat_history" as input variables
agent_prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "chat_history", "agent_scratchpad"],
)

# Create an instance of ZeroShotAgent with the LLMChain and the allowed tool names
zero_shot_agent = ZeroShotAgent(
    llm_chain=LLMChain(llm=llm, prompt=agent_prompt),
    allowed_tools=[tool.name for tool in tools]
)

# Initiate memory which allows for storing and extracting messages
memory = ConversationBufferMemory(memory_key="chat_history")

# Create an AgentExecutor with memory parameter
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=zero_shot_agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory
)

# Define initial question as user input
user_inquiry = "Which are most popular the genres?"

# Run the agent to generate a response
agent_executor.run(user_inquiry)

# Define follow-up question as user input
user_inquiry = "Can you elaborate more?"

# Run the agent to generate another response
agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
Thought: To know the most popular genres, we need to count the number of tracks per genre. In order to do this, I need to use the Analytics tool.
Action: Analytics
Action Input: SELECT genre, COUNT(*) FROM tracks GROUP BY genre ORDER BY COUNT(*) DESC;

> Entering new AgentExecutor chain...
Question: What are the counts of tracks by genre?
Thought: I need to execute a SQL query to get the count of tracks for each genre.
Action: Analytics
Action Input: SELECT genre, COUNT(*) FROM tracks GROUP BY genre ORDER BY COUNT(*) DESC;

> Entering new AgentExecutor chain...
Question: What is the count of tracks per genre?
Thought: I need to use SQL to find the number of tracks per genre.
Action: Analytics
Action Input: SELECT genre, COUNT(*) FROM tracks GROUP BY genre ORDER BY COUNT(*) DESC;

> Entering new SQL Agent Executor chain...
The question is a SQL query. The SQL query is trying to get a count of tracks per genre, and then order the genres by the numb

'Could you please specify what you would like me to elaborate on?'

: 